In [11]:
from IPython import get_ipython
get_ipython().run_line_magic('clear', '')
get_ipython().run_line_magic('reset', '-f')

import numpy as np
import time
import json
import h5py
import scipy.sparse as sparse
import scipy.sparse.linalg as linalg
import scipy as sp

#from solver_class import FiniteElementCylinderSolver, ReducedBasisCylinderSolver
from FreeFEM import FFmatrix_fread, FFvector_fread

In [14]:
# Load coordinates:
coordinates_Px = FFvector_fread('mesh_utilities/vv_cc_Px.btxt')
coordinates_Pq = FFvector_fread('mesh_utilities/vv_cc_Pq.btxt')

# Identify degrees of freedom of the Px and Pq spaces:
sFO_Px = coordinates_Px.shape[0]
sFO_Pq = coordinates_Pq.shape[0]
print(rf'Degree of freedom = {sFO_Px}')

# Define vector of all Px indeces:
mask_all = np.arange(sFO_Px, dtype=int)

# Identify indeces of the top, bottom and middle boundaries in Px:
mask_inf_bc = mask_all[np.isclose([coordinates_Px[ii, 1] for ii in range(sFO_Px)], np.zeros((sFO_Px)))]
mask_sup_bc = mask_all[np.isclose([coordinates_Px[ii, 1] for ii in range(sFO_Px)], np.ones((sFO_Px)))]
mask_mid_bc = mask_all[np.isclose([coordinates_Px[ii, 0] for ii in range(sFO_Px)], np.zeros((sFO_Px)))]
mask_ext_bc = mask_all[np.isclose([coordinates_Px[ii, 0] for ii in range(sFO_Px)], np.ones((sFO_Px)))]

# Identify indeces of the Dirichlet b.c. for the temperature and concentration field:
mask_bc_T = np.fromiter(set(mask_sup_bc), int)
mask_bc_C = np.fromiter(set(mask_sup_bc) | set(mask_ext_bc) | set(mask_inf_bc), int)

# Identify indeces of the interior for the temperature and concentration field:
mask_in_T = [ii for ii in mask_all if ii not in mask_bc_T]
mask_in_C = [ii for ii in mask_all if ii not in mask_bc_C]

# Import mass matrices on the Px and on the Pq space:
mass_Px = FFmatrix_fread('mesh_utilities/ww_mm_Px.btxt')
mass_Pq = FFmatrix_fread('mesh_utilities/ww_mm_Pq.btxt')
mass_Pm = FFmatrix_fread('mesh_utilities/ww_uu_Px_Pq.btxt')

# Extract integration weights and diagonal matrix with their inverse:
weights_Pq = mass_Pq.diagonal()
project_Pq = sparse.diags(np.reciprocal(weights_Pq))

# Import map from Px to Pq and to the dx, dy, dz derivative evaluated in Pq:
PxtoPquu_C = project_Pq.dot(FFmatrix_fread('mesh_utilities/ww_uu_Px_Pq.btxt')[:, mask_in_C])
PxtoPqdx_C = project_Pq.dot(FFmatrix_fread('mesh_utilities/ww_dx_Px_Pq.btxt')[:, mask_in_C])
PxtoPqdy_C = project_Pq.dot(FFmatrix_fread('mesh_utilities/ww_dy_Px_Pq.btxt')[:, mask_in_C])

# Define weights for the integration in polar coordinates:
polar_weights_Pq = np.multiply(coordinates_Pq[:, 0], weights_Pq)

# Compute volume of the domain:
volume = np.dot(np.ones(sFO_Pq), polar_weights_Pq)

# Assemble mass matrix and projected forcing term:
forc_C = PxtoPquu_C.T.dot(polar_weights_Pq[:, None])
mass_C = PxtoPquu_C.T.dot(PxtoPquu_C.multiply(polar_weights_Pq[:, None]))
inte_C = PxtoPquu_C.T.dot(polar_weights_Pq) / volume

Degree of freedom = 10201


In [15]:
# Lettura dei parametri dal file JSON
with open("reference case/Reference_parameters.json", "r") as json_file:
    parameters = json.load(json_file)
FISSION_ENERGY = parameters["FISSION_ENERGY"]                       
FISSION_RATE = parameters["FISSION_RATE"]                   
RADIUS = parameters["RADIUS"]
LENGTH = parameters["LENGTH"]
FISSION_YIELD = parameters["FISSION_YIELD"]                 
FUEL_THERMAL_CONDUCTIVITY = parameters["FUEL_THERMAL_CONDUCTIVITY"]
T_BC = parameters["T_BC"]  
TIME_FINAL = 1.00E+07
TIME_DELTA = 1.00E+04

In [16]:
class FiniteElementCylinderSolver(object):

    # Load the Y and X mass matricess, the list of partial stiffness matricess, the forcing term and the averaging operator:
    YY_FO = np.load('model_files/YY_FO.npy', allow_pickle=True)[0]
    XX_FO = np.load('model_files/XX_FO.npy', allow_pickle=True)[0]
    KK_FO = np.load('model_files/KK_FO.npy', allow_pickle=True)
    FF_FO = np.load('model_files/FF_FO.npy')
    AA_FO = np.load('model_files/AA_FO.npy')

    # Load the interpolation matrix, the interpolation coordinates, and the vecotr of coercivity constants:
    II_FO = np.load('model_files/II_FO.npy')
    ZZ_FO = np.load('model_files/ZZ_FO.npy')
    CC_FO = np.load('model_files/CC_FO.npy')
    
    # Define full order problem size and affine decomposition size:
    sFO = YY_FO.shape[0]
    sHR = len(KK_FO)

    # Define the solver class clonstructor:
    def __init__(self):

        # Define problem size:        
        self.basis = None
        self.sRB = None

    # Define method assembling left-hand-side and right-hand-side:
    def assemble(self, RADIUS=1E-5, LENGTH=1E-3, F_RATE=3E+19, F_YIELD=0.24, F_ENERGY = 3.215e-11, T_BC=2E+03, C_BC=0.0, KTH = 2.208, dt=1E+4):

        # Define reference sources intensity:
        self.sgroup_T = (F_RATE * F_ENERGY * LENGTH**2)/(KTH)
        self.source_C = F_RATE * F_YIELD 

        # Define heat and mass diffusion coefficiets:
        self.alpha_C = 7.60E-10 * np.exp(- 4.86E-19 / (T_BC + self.sgroup_T * (1 - self.ZZ_FO**2) / 2) / 1.380649E-23) +\
                       5.64E-25 * np.exp(- 1.91E-19 / (T_BC + self.sgroup_T * (1 - self.ZZ_FO**2) / 2) / 1.380649E-23) * np.sqrt(F_RATE) +\
                       8.00E-40 * F_RATE
        
        # Compute interpolation vector and bound to the problem coercivity:
        self.inter = np.hstack((self.II_FO @ self.alpha_C / RADIUS**2, self.II_FO @ self.alpha_C / LENGTH**2))
        self.coerc = self.inter @ self.CC_FO
        self.force = self.source_C * self.FF_FO

        # Assemble the stiffnes term for this parameter configuration:
        self.SS_FO = self.inter @ self.KK_FO
        
        # Assemble left-hand-side and right-hand side (static part):
        self.LHS_static = self.XX_FO + dt * self.SS_FO
        self.RHS_static = dt * self.force
        self.SPLU = linalg.splu(self.LHS_static)

        # Save time delta:
        self.dt = dt

    #  Define solver to solve the problem over a given number of iterations:
    def solve(self, n_steps=1000, verbose=False):

        # Iteratively solve the problem:
        solution = np.zeros((self.sFO, n_steps+1))
        for ii in range(n_steps):
            #solution[:, ii+1], _ = linalg.bicgstab(self.LHS_static, self.RHS_static + self.XX_FO.dot(solution[: , ii:ii+1]))
            solution[:, ii+1] = self.SPLU.solve(self.RHS_static[:, 0] + self.XX_FO.dot(solution[: , ii]))

            # Print current iteration and average temperature and concentration:
            if verbose:
                print('Current time:', (ii+1) * self.dt, 's')
                print('Average concentration:', self.AA_FO @ solution[:, ii+1], 'atm/m^3\n')

        # Return solution:
        return solution

    # Define a method to compute the squared norm of a solution:
    def solution_energy(self, solution):
        return self.dt * np.einsum('ik,ik', self.SS_FO @ solution, solution) + self.XX_FO.dot(solution[:, -1]).dot(solution[:, -1])

    # Get the full order representation of the state:
    def get_FO(self, solution):
        return solution

    # Get the full order representation of the state:
    def get_RB(self, solution):
        return solution

In [21]:
solver_FO = FiniteElementCylinderSolver()
solver_FO.assemble(F_YIELD=FISSION_YIELD, F_RATE=FISSION_RATE, T_BC=T_BC, LENGTH=LENGTH, RADIUS=RADIUS, dt=TIME_DELTA)
t0 = time.time()
for ii in range(1):
    solution_FO = solver_FO.solve(n_steps=1000, verbose=False)
time_FO = (time.time() - t0)/1
print(rf'FO elapsed time = {time_FO:.5f} seconds')
print(f'Average concentration FOM: {inte_C @ solution_FO[:, -1]}')

FO elapsed time = 0.91701 seconds
Average concentration FOM: 1.6440831717148707e+24


In [6]:
class ReducedBasisCylinderSolver(object):
    
    # Load the mass matrix, the list of partial stiffness matricess, the forcing term and the averaging operator:
    YY_FO = np.load('model_files/YY_FO.npy', allow_pickle=True)[0]
    XX_FO = np.load('model_files/XX_FO.npy', allow_pickle=True)[0]
    KK_FO = np.load('model_files/KK_FO.npy', allow_pickle=True)
    FF_FO = np.load('model_files/FF_FO.npy')
    AA_FO = np.load('model_files/AA_FO.npy')

    # Load the interpolation matrix, the interpolation coordinates, and the vecotr of coercivity constants:
    II_FO = np.load('model_files/II_FO.npy')
    ZZ_FO = np.load('model_files/ZZ_FO.npy')
    CC_FO = np.load('model_files/CC_FO.npy')
    
    # Define full order problem size and affine decomposition size:
    sFO = YY_FO.shape[0]
    sHR = len(KK_FO)
    
    # Define the solver class clonstructor:
    def __init__(self, basis):

        # Save basis functions and reduced basis space size:
        self.basis = basis
        self.sRB = basis.shape[1]

        # Reduce Y mass matrix:
        self.YY_RB = basis.T @ self.YY_FO @ basis
        self.AA_RB = basis.T @ self.AA_FO
        
        # Assemble Rietz representer and reduced basis projection of the forcing term:
        self.FF_rietz = linalg.spsolve(self.YY_FO, self.FF_FO)
        self.FF_RB    = basis.T @ self.FF_FO

        # Assemble Rietz representer and reduced basis projection of the mass matrix:
        self.XX_right = self.XX_FO @ basis
        self.XX_rietz = linalg.spsolve(self.YY_FO, self.XX_right)
        self.XX_RB    = basis.T @ self.XX_right

        # Assemble Rietz representer and reduced basis projection of the partial stiffness matrix:
        self.KK_right = np.array([self.KK_FO[ii] @ basis for ii in range(self.sHR)])
        self.KK_rietz = np.array([linalg.spsolve(self.YY_FO, self.KK_right[ii]) for ii in range(self.sHR)])
        self.KK_RB    = np.array([basis.T @ self.KK_right[ii] for ii in range(self.sHR)])

        # Pre-assemble the scalar and vector terms for the residual energy computation:
        self.TT_pr_bb = self.FF_rietz.T @ self.FF_FO
        self.TT_pr_bm = self.FF_rietz.T @ self.XX_right 
        self.TT_pr_mm = self.XX_rietz.T @ self.XX_right

        # Pre-assemble all the tensor terms for the residual energy computation:
        self.TT_pr_ab = np.array( [self.KK_right[ii, :, :].T @ self.FF_rietz for ii in range(self.sHR)])
        self.TT_pr_am = np.array( [self.KK_right[ii, :, :].T @ self.XX_rietz for ii in range(self.sHR)])
        self.TT_pr_aa = np.array([[self.KK_right[ii, :, :].T @ self.KK_rietz[jj, :, :] for ii in range(self.sHR)] for jj in range(self.sHR)])
    
    # Define method assembling left-hand-side and right-hand-side:
    def assemble(self, RADIUS=1E-5, LENGTH=1E-3, F_RATE=3E+19, F_YIELD=0.24, F_ENERGY = 3.215e-11, T_BC=2E+03, C_BC=0.0, KTH = 2.208, dt=1E+4):

        # Define reference sources intensity:
        self.sgroup_T = (F_RATE * F_ENERGY * LENGTH**2)/(KTH) 
        self.source_C = F_RATE * F_YIELD 

        # Define heat and mass diffusion coefficiets:
        self.alpha_C = 7.60E-10 * np.exp(- 4.86E-19 / (T_BC + self.sgroup_T * (1 - self.ZZ_FO**2) / 2) / 1.380649E-23) +\
                       5.64E-25 * np.exp(- 1.91E-19 / (T_BC + self.sgroup_T * (1 - self.ZZ_FO**2) / 2) / 1.380649E-23) * np.sqrt(F_RATE) +\
                       8.00E-40 * F_RATE
        
        # Compute interpolation vector and bound to the problem coercivity:
        self.inter = np.hstack((self.II_FO @ self.alpha_C / RADIUS**2, self.II_FO @ self.alpha_C / LENGTH**2))
        self.coerc = self.inter @ self.CC_FO
        self.force = self.source_C * self.FF_RB

        # Assemble the scalar and vector terms for the residual energy computation:
        self.DD_pr_bb = self.TT_pr_bb * (self.source_C * self.source_C)
        self.DD_pr_bm = self.TT_pr_bm * (- 2 * self.source_C / dt)
        self.DD_pr_mm = self.TT_pr_mm * (  1 / (dt * dt))

        # Assemble matrices for the computation of the residual energy for the current parameter configuration:
        self.DD_pr_ab = np.tensordot(self.inter, self.TT_pr_ab, axes=[0, 0]) * (-2 * self.source_C)
        self.DD_pr_am = np.tensordot(self.inter, self.TT_pr_am, axes=[0, 0]) * ( 2 / dt)
        self.DD_pr_aa = np.tensordot(self.inter, np.tensordot(self.inter, self.TT_pr_aa, axes=[0, 0]), axes=[0, 0])

        # Assemble reduced basis stiffnes matrix for this parameter configuration:
        self.SS_RB = np.tensordot(self.inter, self.KK_RB, axes=[0, 0])
        
        # Assemble left-hand-side and right-hand side (static part):
        self.LHS_static = self.XX_RB + dt * self.SS_RB
        self.RHS_static = dt * self.force
        self.LU, self.PIV = sp.linalg.lu_factor(self.LHS_static)

        # Save time delta:
        self.dt = dt

    #  Define reduced basis solver to solve the problem over a given number of iterations:
    def solve(self, n_steps=1000, verbose=False, error_bound=True):

        # Iteratively solve the problem:
        solution = np.zeros((self.sRB, n_steps+1))
        for ii in range(n_steps):
            solution[:, ii+1] = sp.linalg.lu_solve((self.LU, self.PIV), self.RHS_static[:, 0] + self.XX_RB.dot(solution[: , ii]))

            # Print current iteration and average temperature and concentration:
            if verbose:
                print('Current time:', (ii+1) * self.dt, 's')
                print('Average concentration:', self.AA_RB @ solution[:, ii+1], 'atm/m^3\n')

        # Return solution and in case the error bound:
        if error_bound: return solution, np.sqrt(self.residual_energy(solution) / self.solution_energy(solution) / self.coerc)
        return solution

    # Define a method to compute the squared norm of a solution:
    def solution_energy(self, solution):
        return self.dt * np.einsum('ik,ik', self.SS_RB @ solution, solution) + self.XX_RB.dot(solution[:, -1]).dot(solution[:, -1])

    # Define a method to compute the squared norm of the residual of a solution:
    def residual_energy(self, solution):

        # Compute solution increments:
        increment = solution[:, 1:] - solution[:, :-1]

        # Rreturn squared residual norm:
        return self.dt * (np.einsum('ik,ik', self.DD_pr_am @ increment, solution[:, 1:])       + np.sum(self.DD_pr_bm @ increment) +\
                          np.einsum('ik,ik', self.DD_pr_aa @ solution[:, 1:], solution[:, 1:]) + np.sum(self.DD_pr_ab @ solution[:, 1:]) +\
                          np.einsum('ik,ik', self.DD_pr_mm @ increment, increment)             + self.DD_pr_bb[0] * increment.shape[1] )

    # Get the full order representation of the reduced basis state:
    def get_FO(self, solution):
        return self.basis @ solution

    # Get the reduced basis representation of the full order state:
    def get_RB(self, solution):
        return np.linalg.solve(self.XX_RB, self.XX_right.T @ solution)

In [10]:
# Apri il file in modalità lettura
with h5py.File('new algorithm/checkpoint/step_190/matrici_RB.h5', 'r') as hf:
    basis = hf['basis'][:,:]  
print(basis.shape)
solver_RB = ReducedBasisCylinderSolver(basis[:, :])
solver_RB.assemble(F_YIELD=FISSION_YIELD, F_RATE=FISSION_RATE, T_BC=T_BC, LENGTH=LENGTH, RADIUS=RADIUS, dt=TIME_DELTA)
t0 = time.time()
for ii in range(100):
    solution_RB = solver_RB.solve(n_steps=1000, error_bound=False)
time_RB =(time.time() - t0)/100
print(rf'RB elapsed time = {time_RB:.5f}')
print(solution_RB.shape)

(9900, 190)
RB elapsed time = 0.06456
(190, 1001)


In [12]:
ratio = time_RB/time_FO
print(rf'Ratio = {ratio}')
inverse_ratio = 1 / ratio
print(rf'Ratio = 1/{inverse_ratio:.5f}')

Ratio = 0.09337127915938505
Ratio = 1/10.70993


In [14]:
class ReducedBasisCylinderSolver(object):
    # Load the mass matrix, the list of partial stiffness matricess, the forcing term and the averaging operator:
    YY_FO = np.load('model_files/YY_FO.npy', allow_pickle=True)[0]
    XX_FO = np.load('model_files/XX_FO.npy', allow_pickle=True)[0]
    KK_FO = np.load('model_files/KK_FO.npy', allow_pickle=True)
    FF_FO = np.load('model_files/FF_FO.npy')
    AA_FO = np.load('model_files/AA_FO.npy')
 
    # Load the interpolation matrix, the interpolation coordinates, and the vecotr of coercivity constants:
    II_FO = np.load('model_files/II_FO.npy')
    ZZ_FO = np.load('model_files/ZZ_FO.npy')
    CC_FO = np.load('model_files/CC_FO.npy')
    # Define full order problem size and affine decomposition size:
    sFO = YY_FO.shape[0]
    sHR = len(KK_FO)
    # Define the solver class clonstructor:
    def __init__(self, basis):
 
        # Save basis functions and reduced basis space size:
        self.basis = basis
        self.sRB = basis.shape[1]
 
        # Reduce Y mass matrix:
        self.YY_RB = basis.T @ self.YY_FO @ basis
        self.AA_RB = basis.T @ self.AA_FO
        # Assemble Rietz representer and reduced basis projection of the forcing term:
        self.FF_rietz = linalg.spsolve(self.YY_FO, self.FF_FO)
        self.FF_RB    = basis.T @ self.FF_FO
 
        # Assemble Rietz representer and reduced basis projection of the mass matrix:
        self.XX_right = self.XX_FO @ basis
        self.XX_rietz = linalg.spsolve(self.YY_FO, self.XX_right)
        self.XX_RB    = basis.T @ self.XX_right
 
        # Assemble Rietz representer and reduced basis projection of the partial stiffness matrix:
        self.KK_right = np.array([self.KK_FO[ii] @ basis for ii in range(self.sHR)])
        self.KK_rietz = np.array([linalg.spsolve(self.YY_FO, self.KK_right[ii]) for ii in range(self.sHR)])
        self.KK_RB    = np.array([basis.T @ self.KK_right[ii] for ii in range(self.sHR)])
 
        # Pre-assemble the scalar and vector terms for the residual energy computation:
        self.TT_pr_bb = self.FF_rietz.T @ self.FF_FO
        self.TT_pr_bm = self.FF_rietz.T @ self.XX_right 
        self.TT_pr_mm = self.XX_rietz.T @ self.XX_right
 
        # Pre-assemble all the tensor terms for the residual energy computation:
        self.TT_pr_ab = np.array( [self.KK_right[ii, :, :].T @ self.FF_rietz for ii in range(self.sHR)])
        self.TT_pr_am = np.array( [self.KK_right[ii, :, :].T @ self.XX_rietz for ii in range(self.sHR)])
        self.TT_pr_aa = np.array([[self.KK_right[ii, :, :].T @ self.KK_rietz[jj, :, :] for ii in range(self.sHR)] for jj in range(self.sHR)])
    # Define method assembling left-hand-side and right-hand-side:
    def assemble(self, RADIUS=1E-5, LENGTH=1E-3, F_RATE=3E+19, F_YIELD=0.24, F_ENERGY = 3.215e-11, T_BC=2E+03, C_BC=0.0, KTH = 2.208, dt=1E+4):
 
        # Define reference sources intensity:
        self.sgroup_T = (F_RATE * F_ENERGY * LENGTH**2)/(KTH) 
        self.source_C = F_RATE * F_YIELD
 
        # Define heat and mass diffusion coefficiets:
        self.alpha_C = 7.60E-10 * np.exp(- 4.86E-19 / (T_BC + self.sgroup_T * (1 - self.ZZ_FO**2) / 2) / 1.380649E-23) +\
                       5.64E-25 * np.exp(- 1.91E-19 / (T_BC + self.sgroup_T * (1 - self.ZZ_FO**2) / 2) / 1.380649E-23) * np.sqrt(F_RATE) +\
                       8.00E-40 * F_RATE
        # Compute interpolation vector and bound to the problem coercivity:
        self.inter = np.hstack((self.II_FO @ self.alpha_C / RADIUS**2, self.II_FO @ self.alpha_C / LENGTH**2))
        self.coerc = self.inter @ self.CC_FO
        self.force = self.source_C * self.FF_RB
 
        # Assemble the scalar and vector terms for the residual energy computation:
        self.DD_pr_bb = self.TT_pr_bb * (self.source_C * self.source_C)
        self.DD_pr_bm = self.TT_pr_bm * (- 2 * self.source_C / dt)
        self.DD_pr_mm = self.TT_pr_mm * (  1 / (dt * dt))
 
        # Assemble matrices for the computation of the residual energy for the current parameter configuration:
        self.DD_pr_ab = np.tensordot(self.inter, self.TT_pr_ab, axes=[0, 0]) * (-2 * self.source_C)
        self.DD_pr_am = np.tensordot(self.inter, self.TT_pr_am, axes=[0, 0]) * ( 2 / dt)
        self.DD_pr_aa = np.tensordot(self.inter, np.tensordot(self.inter, self.TT_pr_aa, axes=[0, 0]), axes=[0, 0])
 
        # Assemble reduced basis stiffnes matrix for this parameter configuration:
        self.SS_RB = np.tensordot(self.inter, self.KK_RB, axes=[0, 0])
        # Assemble left-hand-side and right-hand side (static part):
        self.LHS_static = self.XX_RB + dt * self.SS_RB
        self.RHS_static = dt * self.force
        t_x = time.time()
        self.LU, self.PIV = sp.linalg.lu_factor(self.LHS_static)
        print(time.time() - t_x)
 
        # Save time delta:
        self.dt = dt
 
    #  Define reduced basis solver to solve the problem over a given number of iterations:
    def solve(self, n_steps=1000, verbose=False, error_bound=True):
 
        # Iteratively solve the problem:
        solution = np.zeros((self.sRB, n_steps+1))
        for ii in range(n_steps):
            solution[:, ii+1] = sp.linalg.lu_solve((self.LU, self.PIV), self.RHS_static[:, 0] + self.XX_RB.dot(solution[: , ii]))
 
            # Print current iteration and average temperature and concentration:
            if verbose:
                print('Current time:', (ii+1) * self.dt, 's')
                print('Average concentration:', self.AA_RB @ solution[:, ii+1], 'atm/m^3\n')
 
        # Return solution and in case the error bound:
        if error_bound: return solution, np.sqrt(self.residual_energy(solution) / self.solution_energy(solution) / self.coerc)
        return solution
 
    # Define a method to compute the squared norm of a solution:
    def solution_energy(self, solution):
        return self.dt * np.einsum('ik,ik', self.SS_RB @ solution, solution) + self.XX_RB.dot(solution[:, -1]).dot(solution[:, -1])
 
    # Define a method to compute the squared norm of the residual of a solution:
    def residual_energy(self, solution):
 
        # Compute solution increments:
        increment = solution[:, 1:] - solution[:, :-1]
 
        # Rreturn squared residual norm:
        return self.dt * (np.einsum('ik,ik', self.DD_pr_am @ increment, solution[:, 1:])       + np.sum(self.DD_pr_bm @ increment) +\
                          np.einsum('ik,ik', self.DD_pr_aa @ solution[:, 1:], solution[:, 1:]) + np.sum(self.DD_pr_ab @ solution[:, 1:]) +\
                          np.einsum('ik,ik', self.DD_pr_mm @ increment, increment)             + self.DD_pr_bb[0] * increment.shape[1] )
 
    # Get the full order representation of the reduced basis state:
    def get_FO(self, solution):
        return self.basis @ solution
 
    # Get the reduced basis representation of the full order state:
    def get_RB(self, solution):
        return np.linalg.solve(self.XX_RB, self.XX_right.T @ solution)

In [15]:
print('first')
solver_RB = ReducedBasisCylinderSolver(np.zeros((basis.shape[0], 40)))
for ii in range(1):
    solver_RB.assemble()
    tx = time.time()
    a,b = sp.linalg.lu_factor(solver_RB.LHS_static)
    print(time.time()-tx)

print('second')
 
solver_RB = ReducedBasisCylinderSolver(np.zeros((basis.shape[0], 173)))
for ii in range(1):
    solver_RB.assemble()
    tx = time.time()
    a,b = sp.linalg.lu_factor(solver_RB.LHS_static)
    print(time.time()-tx)

first


/var/folders/gs/p2p59ns16hlgpqv_027665gm0000gn/T/ipykernel_5937/3653025618.py:81: LinAlgWarning: Diagonal number 1 is exactly zero. Singular matrix.
  self.LU, self.PIV = sp.linalg.lu_factor(self.LHS_static)
/var/folders/gs/p2p59ns16hlgpqv_027665gm0000gn/T/ipykernel_5937/2405499725.py:6: LinAlgWarning: Diagonal number 1 is exactly zero. Singular matrix.
  a,b = sp.linalg.lu_factor(solver_RB.LHS_static)


0.00013494491577148438
0.00016689300537109375
second
0.00015306472778320312
0.0006511211395263672


/var/folders/gs/p2p59ns16hlgpqv_027665gm0000gn/T/ipykernel_5937/2405499725.py:15: LinAlgWarning: Diagonal number 1 is exactly zero. Singular matrix.
  a,b = sp.linalg.lu_factor(solver_RB.LHS_static)
